<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Thread-Versus-Coroutine:-A-Comparison" data-toc-modified-id="Thread-Versus-Coroutine:-A-Comparison-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Thread Versus Coroutine: A Comparison</a></span></li></ul></div>

##### Thread Versus Coroutine: A Comparison

spinner with thread

In [4]:
import threading
import itertools
import time
import sys


class Signal: #1 Signal类有一个可变成员 go, 用它来控制线程的开启或者关闭.
    go = True


def spin(msg, signal): #2 第二个参数是 Signal的实例
    write, flush = sys.stdout.write, sys.stdout.flush
    for char in itertools.cycle('|/-\\'): #3 这是一个无限循环, 参考 itertools.cycle
        status = char + '' + msg
        write(status)
        flush()
        write('\x08' * len(status)) #4 The trick to do text-mode animation: move the cursor back with backspace characters (\x08).
                                    #然而在jupyter里面看不到......
        time.sleep(.1)
        if not signal.go: #5 If the go attribute is no longer True, exit the loop.
            break
    write(' ' * len(status) + '\x08' * len(status)) #6 Clear the status line by overwriting with spaces and 
                                                    #  moving the cursor back to the beginning.


def slow_function(): #7 模拟一个耗时的计算
    time.sleep(3) #8 sleep函数阻塞主线程, 同时使主线程释放GIL, 此时副线程得以运行.
    return '高科技'


def supervisor(): #9 开辟副线程, 运行副线程并且模拟耗时计算, 最后杀死线程
    signal = Signal()
    # 开一个线程
    spinner = threading.Thread(target=spin, args=('thinking', signal))
    print('spinner object:', spinner) #10
    spinner.start() #11
    result = slow_function() #12
    signal.go = False #13
    spinner.join() #14
    return result


def main():
    result = supervisor() #15
    print('Answer:', result)

In [5]:
main()

spinner object: <Thread(Thread-5, initial)>
|thinkin/thinkin-thinkin\thinkin|thinkin/thinkin-thinkin\thinkin|thinkin/thinkin-thinkin\thinkin|thinkin/thinkin-thinkin\thinkin|thinkin/thinkin-thinkin\thinkin|thinkin/thinkin-thinkin\thinkin|thinkin/thinkin-thinkin\thinkin|thinkin        Answer: 高科技


spinner with asyncio

In [7]:
import asyncio
import itertools
import sys


@asyncio.coroutine #1
def spin(msg): #2
    write, flush = sys.stdout.write, sys.stdout.flush
    for char in itertools.cycle('|/-\\'):
        status = char + ' ' + msg
        write(status)
        flush()
        write('\x08' * len(status))
        try:
            yield from asyncio.sleep(.1) #3
        except asyncio.CancelledError: #4
            break
    write(' ' * len(status) + '\x08' * len(status))


@asyncio.coroutine
def slow_function(): #5
    yield from asyncio.sleep(3) #6
    return '高科技'


@asyncio.coroutine
def supervisor(): #7
    spinner = asyncio.async(spin('thinking')) #8
    print('spinner object:', spinner) #9
    result = yield from slow_function() #10
    spinner.cancel() #11
    return result


def main():
    loop = asyncio.get_event_loop() #12
    result = loop.run_until_complete(supervisor()) #13
    loop.close()
    print('Answer:', result)

<center>总结</center>

1. 如果你想使用 asyncio 来实现协程, 最好加上 @asyncio.coroutine 装饰器.
2. 这里不需要外界提供一个信号来控制协程.
3. 使用 asyncio 来实现协程时, IO-bound的操作要小心, 为了不阻塞事件循环, 需要用 yield from asyncio.sleep(.1).
4. 如果抛出 asyncio.CancelledError 异常, 就退出循环, 协程中止.
5. slow_function 现在也是一个协程(因为他原本有IO-bound的操作).
6. The yield from asyncio.sleep(3) expression handles the control flow to the main loop, 
which will resume this coroutine after the sleep delay.
7. supervisor 现在也是一个协程函数, 内部会 通过 yield from 调用协程函数 slow_function.
8. asyncio.async(…) schedules the spin coroutine to run, wrapping it in a ***Task*** object, which is ***returned immediately***.
9. 打印 Task 对象.
10. yield from slow_function(), 并获得其返回值. 同时 时间循环将会继续运行, 这是因为 slow_fucntion 内部 yield from 了 asyncio.sleep(3), 控制权会立刻回到主循环.
11. 通过调用 Task 对象的 cancell 方法, raise asyncio.CancelledError, 抛出异常的地方是Task包装的协程函数**当前挂起的 yield 语句的位置**. 协程函数可以捕获异常, 延迟异常甚至拒绝异常.
12. 获得事件循环的***引用***.
13. 启动 supervisor 协程函数直到他终止并且获取其返回值.